In [ ]:
import os
os.chdir("../../..")
import deepof.data

In [ ]:
%load_ext autoreload
%autoreload 2

# Formatting your data: feature extraction from motion tracking output

##### Tutorial index:
    
* Create and run a project.
* Load a previously generated project.
* Interact with your project: generate coordinates, distances, angles, and areas.
* Exploratory visualizations: heatmaps, basic embeddings, and processed animations.
* Data preprocessing: generate datasets for machine learning.

### Create and run a project

Project creation should be more automatic. Specify a project path, a video and a tab path. Running the project should create the needed directories automatically, and store any further files in place to keep everything tidy! Shouldn't be much work.

In [ ]:
# Create and run a project. Explore different processing parameters (smoothing, imputation)

In [ ]:
my_deepof_project = deepof.data.Project(
                project_path=os.path.join("../../../NatComm_trained_models/"),
                video_path=os.path.join("../../../NatComm_trained_models/deepof_SI_project/Videos/"),
                table_path=os.path.join("../../../NatComm_trained_models/deepof_SI_project/Tables/"),
                project_name="deepof_SI_project_manual",
                arena="circular-manual",
                animal_ids=["B", "W"],
                video_format=".mp4",
                exclude_bodyparts=["Tail_1", "Tail_2", "Tail_tip"],
                video_scale=380,
                enable_iterative_imputation=10,
)

In [ ]:
my_deepof_project = my_deepof_project.create()

### Load a previously generated project

In [ ]:
# Load a previously saved project
my_deepof_project = deepof.data.load_project("../../Desktop/deepOF_CSDS_tutorial_dataset/deepof_tutorial/")

In [ ]:
import networkx as nx
tt = my_deepof_project.get_graph_dataset(
    animal_id="mouse_black_tail"
)
my_deepof_project.deep_unsupervised_embedding(tt[0], nx.adjacency_matrix(tt[1]).todense(), epochs=1)

### Interact with your project: generate coordinates, distances, angles, and areas.

In [ ]:
# Get and plot coordinates, distances, angles, and areas. Try and visualizing different centering and rotations

In [ ]:
my_deepof_project.get_angles()['20191204_Day2_SI_JB08_Test_54']

In [ ]:
my_deepof_project.get_coords(polar=True)['20191204_Day2_SI_JB08_Test_54']

In [ ]:
my_deepof_project.deep_unsupervised_embedding()

In [ ]:
my_deepof_project.get_distances()['20191204_Day2_SI_JB08_Test_54']

In [ ]:
my_deepof_project.get_angles()['20191204_Day2_SI_JB08_Test_54']

In [ ]:
my_deepof_project.get_areas()

In [ ]:
my_deepof_project.get_areas()['20191204_Day2_SI_JB08_Test_54']

In [ ]:
# Show how to merge different feature objects

my_deepof_project.get_coords().pca()

### Exploratory visualizations: heatmaps, basic embeddings, and processed animations.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import deepof.visuals
from matplotlib.patches import Ellipse

sns.set_context("talk")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

deepof.visuals.plot_heatmaps(
    my_deepof_project, 
    ["B_Nose"],
    center="arena", 
    exp_condition="Nonstressed",
    ax=ax1,
    show=False,
    display_arena=True,
    experiment_id="average",
)

deepof.visuals.plot_heatmaps(
    my_deepof_project, 
    ["B_Nose"],
    center="arena", 
    exp_condition="Stressed",
    ax=ax2,
    show=False,
    display_arena=True,
    experiment_id="average",
)

plt.tight_layout()
plt.show()

In [ ]:
# Create a preprocessed version of the dataset we're after
pp, _, _ = my_deepof_project.get_graph_dataset(
  #  animal_id="B",
    center="Center",
    align="Spine_1",
    preprocess=True,
    scale="robust",
)

In [ ]:
pp[2]

In [ ]:
from ipywidgets import interact, IntSlider

@interact()
def plot_mouse(i=IntSlider(min=0, max=100)):

    sns.scatterplot(x=pp[0][i][0][:11], y=pp[0][i][0][22:33])
    sns.scatterplot(x=pp[0][i][0][11:22], y=pp[0][i][0][33:44])
#     sns.scatterplot(x=pp[0][15000 + i][0][:11], y=pp[0][15000 + i][0][11:22])
#     sns.scatterplot(x=pp[0][30000 + i][0][:11], y=pp[0][30000 + i][0][11:22])
#     sns.scatterplot(x=pp[0][45000 + i][0][:11], y=pp[0][45000 + i][0][11:22])
#     sns.scatterplot(x=pp[0][60000 + i][0][:11], y=pp[0][60000 + i][0][11:22])
#     sns.scatterplot(x=pp[0][75000 + i][0][:11], y=pp[0][75000 + i][0][11:22])
    
    plt.xlim(-3, 3)
    plt.ylim(-4, 5)
    
    plt.show()

In [ ]:
ss = (pp[0][:25000], pp[1][:25000], pp[2][:25000], pp[3][:25000], pp[4][:25000], pp[5][:25000])

In [ ]:
%%time
cons = my_deepof_project.deep_unsupervised_embedding(
    preprocessed_data,
    embedding_model="VaDE",
    epochs=5,
    encoder_type="TCN",
    n_components=15,
    batch_size=128,
    latent_dim=2,
    kl_warmup=10,
    kl_annealing_mode="linear",
    kmeans_loss=0.0,
    reg_cat_clusters=0.0,
)

In [ ]:
tt = cons.encoder([ss[0], ss[1]])
qq = cons.grouper([ss[0], ss[1]])

In [ ]:
import umap
umap = tt.numpy()#umap.UMAP(n_components=2, n_neighbors=250).fit_transform(tt.numpy())

In [ ]:
means = cons.get_gmm_params['means'].numpy()

sns.scatterplot(x=umap[:, 0], y=umap[:, 1], hue=qq.numpy().argmax(axis=1), palette="tab20")
means = cons.get_layer("grouper").get_layer("gaussian_mixture_latent").c_mu.numpy()
sns.scatterplot(x=means[:,0], y=means[:,1], s=250, c="black")

plt.title("VQVAE embeddings")

plt.legend("")
plt.show()

In [ ]:
from IPython import display

video = deepof.visuals.animate_skeleton(
    my_deepof_project,
    #embedding=emb,
    #cluster_assignments=clust_ass,
    #animal_id="W",
    experiment_id="20191204_Day2_SI_JB08_Test_54",
    frame_limit=250,
    # selected_cluster=0,
    dpi=60,
    # center="Center",
    # align="Spine_1",
    #embedding=[emb_B, emb_W],
    save="SAB_multi_animal_example.mp4",
)

html = display.HTML(video)
display.display(html)
plt.close()

In [ ]:
# Single animal display

In [ ]:
from sklearn.decomposition import PCA

emb = PCA(n_components=3).fit_transform(my_deepof_project.get_coords().filter_id("B")['20191204_Day2_SI_JB08_Test_56'])

In [ ]:
from IPython import display

video = deepof.visuals.animate_skeleton(
    my_deepof_project,
    animal_id="B",
    embedding=emb,
    experiment_id="20191204_Day2_SI_JB08_Test_56",
    frame_limit=750,
    dpi=60,
)

html = display.HTML(video)
display.display(html)
plt.close()

In [ ]:
emb = PCA(n_components=3).fit_transform(my_deepof_project.get_coords(center="Center", align="Spine_1").filter_id("B")['20191204_Day2_SI_JB08_Test_56'])

In [ ]:
from IPython import display

video = deepof.visuals.animate_skeleton(
    my_deepof_project,
    center="Center",
    align="Spine_1",
    animal_id="B",
    embedding=emb,
    experiment_id="20191204_Day2_SI_JB08_Test_56",
    frame_limit=750,
    dpi=60,
)

html = display.HTML(video)
display.display(html)
plt.close()

In [ ]:
# Get and plot coordinates, distances, angles, and areas. Try and visualizing different centering and rotations

In [ ]:
my_deepof_project.get_coords(polar=True)['20191204_Day2_SI_JB08_Test_54']

In [ ]:
my_deepof_project.get_distances()['20191204_Day2_SI_JB08_Test_54']

In [ ]:
my_deepof_project.get_angles()['20191204_Day2_SI_JB08_Test_54']

In [ ]:
my_deepof_project.get_areas()

In [ ]:
my_deepof_project.get_areas()['20191204_Day2_SI_JB08_Test_54']

In [ ]:
# Show how to merge different feature objects

my_deepof_project.get_coords().pca()

### Data preprocessing: generate datasets for machine learning.

In [ ]:
import networkx as nx
from deepof.utils import connect_mouse_topview

G = connect_mouse_topview(animal_ids=["B", "W"], exclude_bodyparts=["Tail_1", "Tail_2", "Tail_tip"])

# Include a (nicer) version of this plot in the tutorial
nx.draw(G, 
        with_labels=True,
        pos=nx.spring_layout(G), 
        node_color=["red" if i.startswith("B") else "blue" for i in G.nodes()])